## K-Nearest Neighbor Algorithmn
K-NN or (KNN) stand for K-Nearest Neighbors is a Non-parametric, supervised Machine Learning  Algorithms mostly used for classification.
The algorithmn uses proximity to make predictions about the grouping of an individual data point.A class label is assigned on the basis of a majority vote in order to solve the classification problems.It's worth noting that the KNN algorithm is also member of a family called 'Lazy learning' models since it only stores a training dataset versus undergoing training stage.This also means that all computations occurs when a classification or prediction is being made. An optimum value of k need to be determined to reduce variance-bias tradeoff.
Choosing the high value of k result to high variance and consumption of processing power while the other way round result to a biased model.
To obtain the optimum k factor the mathematical concepts are applied such as:

##### i) Euclidean distance.

This is the most used distance borrowed directly from Pythogra's Theorem:


$dist(d) = \sqrt{(x-a)^2 + (y-b)^2}$

#### ii) Manhattan distance

Manhattan distance measures the sum of the absolute differences between the coordinates of two points,the method is often used in grid-based systems.

$dist(d) = |(x-a)| + |(y-b)|$

#### iii)Minkowski distance

This methods combine Manhattan and Euclidean distance.

$(\sum _{i=1}^{i=n} |x_{i} -y_{i}|^{p})^ \frac 1 p $

where p is a parameter;

when p = 2 Minkowski distance corresponds to Euclidean distance and when  p = 1 the distance equal to that of the 
Manhattan distance.

####  iV) Hamming distance

This method is particurlaly useful when dealing with categorical or binary data, where each feature can take only two possible values.

$d(x,y) = \sum_{i=1}^{i=n} (xi \space XOR \space yi )$

#### v) Jaccard distance 

This measure dissimilarity between two set that is size of the symmetric differences of the sets divided by the size of their union.

$J(X,Y) = 1 - \frac {|X n Y|} {|XUY|}$

Where ${|X n Y|}$ intersection of set X and Y

Where ${|X U Y|}$ Union of set X and Y

##### Let now roll up our sleeves and get our hands dirty with the Python code:


##### In coding we are going to divide  this into two Sections:
    i) K-NN without Hyperparameter Tunning
    ii) K-NN with Hyperparameter Tunning  

## Part 1: KNN without Hyperparameter Tunning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OrdinalEncoder,LabelEncoder
from sklearn.model_selection  import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("framingham.csv")
type(df)

pandas.core.frame.DataFrame

In [3]:
feat_object= [i for i in df.columns if df[i].dtypes == 'object']
feat_object

['Sex',
 'currentSmoker',
 'BPMeds',
 'prevalentStroke',
 'prevalentHyp',
 'diabetes']

In [4]:
orde =OrdinalEncoder()
ler = LabelEncoder()

orde.fit(df[feat_object])
df[feat_object] = orde.transform(df[feat_object] )

In [5]:
null_col =[i for i in df.columns if df[i].isna().sum()>0 ]
null_col

['education', 'cigsPerDay', 'BPMeds', 'totChol', 'BMI', 'heartRate', 'glucose']

In [6]:
for i in df.columns:
    for j in null_col:
        if j in i:
            print(i)

education
cigsPerDay
BPMeds
totChol
BMI
heartRate
glucose


In [7]:
for i in df.columns:
    k= df[i].isna().sum()
    if k>0:
        df[i] = df[i].replace(np.NaN,value = df[i].median())
        print(df[i].isna().sum())

0
0
0
0
0
0
0


In [8]:
target ='TenYearCHD'
y=df[target]

feat=[i for i in df.columns if 'TenYearCHD' not in i]
X=df[feat]

In [9]:
X = orde.fit_transform(X)
y = ler.fit_transform(y)

In [10]:
from imblearn.over_sampling import SMOTE,RandomOverSampler,ADASYN
oversample = SMOTE(random_state=1)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.30,stratify=y)
X_train,y_train=oversample.fit_resample(X_train,y_train)


In [12]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
x_train_rescaled = scaling.transform(X_train)

In [13]:
model1 =  KNeighborsClassifier(n_neighbors=11,p=5,metric='manhattan')
model1.fit(x_train_rescaled,y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=11, p=5)

In [14]:
y_pred = model1.predict(X_test)

In [15]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[   0 1079]
 [   0  193]]
0.15172955974842767
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1079
           1       0.15      1.00      0.26       193

    accuracy                           0.15      1272
   macro avg       0.08      0.50      0.13      1272
weighted avg       0.02      0.15      0.04      1272



In [16]:
from sklearn.model_selection import GridSearchCV
param_grid = {
#     'n_neighbors'= [1,2],
    'weights':['uniform','distance'],
    'algorithm':['auto','bal_tree','kd_tree','brute'],
    'p':[1,2],
    'metric':['euclidean','manhattan','minkowski','Hamming']
    }


## Part 2: KNN with Hyperparameter Tunning

In [17]:
knn= KNeighborsClassifier(n_neighbors=11)
grid = GridSearchCV(knn,param_grid,cv=5, scoring = 'accuracy')
model = grid.fit(X_train,y_train)

In [18]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [19]:
model.best_params_

{'algorithm': 'auto', 'metric': 'manhattan', 'p': 1, 'weights': 'distance'}

In [20]:
model.best_score_

0.7916500994035786

In [21]:
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[666 413]
 [ 95  98]]
0.60062893081761
              precision    recall  f1-score   support

           0       0.88      0.62      0.72      1079
           1       0.19      0.51      0.28       193

    accuracy                           0.60      1272
   macro avg       0.53      0.56      0.50      1272
weighted avg       0.77      0.60      0.66      1272



In [22]:
best_model = KNeighborsClassifier(algorithm='auto', metric= 'manhattan', p= 2, weights='distance')
best_model.fit(X_test,y_test)

KNeighborsClassifier(metric='manhattan', weights='distance')

In [23]:
y_pre = best_model.predict(X_test)

In [24]:
print("Confusion Matrix:",confusion_matrix(y_test,y_pre))

print("\n Accuracy:", accuracy_score(y_test,y_pre))

print("\n Classification Report:",classification_report(y_test,y_pre))

Confusion Matrix: [[1079    0]
 [   0  193]]

 Accuracy: 1.0

 Classification Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1079
           1       1.00      1.00      1.00       193

    accuracy                           1.00      1272
   macro avg       1.00      1.00      1.00      1272
weighted avg       1.00      1.00      1.00      1272

